In [ ]:
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset
import pandas as pd

pd.options.display.max_columns= None

In [ ]:
dataset = load_dataset("AnikaBasu/CyberbullyingDataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2956 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'text', 'instruction', 'output'],
        num_rows: 2956
    })
})

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df.head()

,input,text,instruction,output
0,,Here is a classification that determines wheth...,"In other words #katandandre, your food was cra...",not_cyberbullying
1,,Here is a classification that determines wheth...,"In other words #katandandre, your food was cra...",not_cyberbullying
2,,Here is a classification that determines wheth...,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
3,,Here is a classification that determines wheth...,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
4,,Here is a classification that determines wheth...,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying


In [ ]:
def drop_columns(df, columns):
    return df.drop(columns=columns, inplace=False)

columns_to_drop = ['input', 'text']
df = drop_columns(df, columns_to_drop)


In [ ]:
df.head()

,instruction,output
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,"In other words #katandandre, your food was cra...",not_cyberbullying
2,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
3,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
4,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying


In [ ]:
df['output'].value_counts()

output
gender                 612
religion               562
other_cyberbullying    483
not_cyberbullying      434
age                    433
ethnicity              432
Name: count, dtype: int64

In [ ]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df

,instruction,output
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,"In other words #katandandre, your food was cra...",not_cyberbullying
2,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
3,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
4,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
...,...,...
2951,U tell ppl its ok if someone ridiculed ur race...,ethnicity
2952,@SexiVixenFromMD you're right the lightskin/da...,ethnicity
2953,"Mad? >>RT @tayyoung_: FUCK OBAMA, dumb ass nigger",ethnicity
2954,@RealKayneWest fuck you you dumb nigger I woul...,ethnicity


In [ ]:
# Assign values to cyberbullying types
cyberbullying_type_mapping = {
    "age": 1,
    "ethnicity": 2,
    "gender": 3,
    "not_cyberbullying": 4,
    "other_cyberbullying": 5,
    "religion": 6
}
df['output'] = df['output'].map(cyberbullying_type_mapping)

In [ ]:
df.head()

,instruction,output
0,"In other words #katandandre, your food was cra...",4
1,"In other words #katandandre, your food was cra...",4
2,Why is #aussietv so white? #MKR #theblock #ImA...,4
3,@XochitlSuckkks a classy whore? Or more red ve...,4
4,"@Jason_Gio meh. :P thanks for the heads up, b...",4


In [ ]:
def preprocess_text(text):

    # Lowercasing
    text = text.lower()

    # Replace links with 'link'
    text = re.sub(r'http\S+|www\S+', 'link', text)

    # Remove mentions (words starting with @)
    text = re.sub(r'@\w+', '', text)

    #remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Removing Punctuation and Special Characters
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Applying text preprocessing to the 'instruction' column in the DataFrame
df['instruction'] = df['instruction'].apply(preprocess_text)

print(df.head())

                                         instruction  output
0                            words food crapilicious       4
1                            words food crapilicious       4
2                                              white       4
3                   classy whore red velvet cupcakes       4
4  meh p thanks heads concerned another angry dud...       4


In [ ]:

# Function to remove emoticons from text
def remove_emoticons(text):
    return re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', '', text)

df['instruction'] = df['instruction'].apply(remove_emoticons)

In [ ]:

#Tokenize data
from nltk.tokenize import word_tokenize

df['tokens'] = df['instruction'].apply(word_tokenize)

# Display the DataFrame with tokenized text
print(df[['instruction', 'tokens']].head())

                                         instruction  \
0                            words food crapilicious   
1                            words food crapilicious   
2                                              white   
3                   classy whore red velvet cupcakes   
4  meh p thanks heads concerned another angry dud...   

                                              tokens  
0                        [words, food, crapilicious]  
1                        [words, food, crapilicious]  
2                                            [white]  
3             [classy, whore, red, velvet, cupcakes]  
4  [meh, p, thanks, heads, concerned, another, an...  


In [ ]:
# NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

# Function to remove stopwords
def remove_stopwords(text):
    return ' '.join(word for word in text.split() if word.lower() not in stop_words)

df['instruction'] = df['instruction'].apply(remove_stopwords)


In [ ]:
df['instruction']

0                                 words food crapilicious
1                                 words food crapilicious
2                                                   white
3                        classy whore red velvet cupcakes
4       meh p thanks heads concerned another angry dud...
                              ...                        
2951    u tell ppl ok someone ridiculed ur race cuz so...
2952    right battle dumb fuck cuz whites see nigger e...
2953                    mad rt fuck obama dumb ass nigger
2954    fuck dumb nigger would never stand fuckin show...
2955    pussy ass bitch dumb muthafuker like fuck u ni...
Name: instruction, Length: 2956, dtype: object

In [ ]:
#Split in train, test and validation
from sklearn.model_selection import train_test_split

# Split data into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
#Making x_train and y_train columns
X_train = train_df['instruction']
y_train = train_df['output']

X_test = test_df['instruction']
y_test = test_df['output']

In [ ]:
data1 = pd.DataFrame(X_train)
data2 = pd.DataFrame(y_train)

data3 = pd.DataFrame(X_test)
data4 = pd.DataFrame(y_test)

In [ ]:
data1.to_csv('X_train')
data2.to_csv('y_train')
data3.to_csv('X_test')
data4.to_csv('y_test')

In [ ]:
#distillBert Model
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import pandas as pd

In [ ]:
!pip install transformers
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Tokenize and encode the text data using DistilBERT
def encode_text(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True, truncation=True, padding=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Apply encoding to the train and test data
X_train_encoded = [encode_text(text) for text in X_train]
X_test_encoded = [encode_text(text) for text in X_test]


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
d1 = pd.DataFrame(X_train_encoded)
d2 = pd.DataFrame(X_test_encoded)

In [ ]:
d1.to_csv('distillBertTrainData_Cyberbullying')
d2.to_csv("distillBertTestData_Cyberbullying")

In [ ]:
# Train Logistic Regression model
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X_train_encoded, y_train)

# Make predictions on train set
logreg_train_preds = logreg_model.predict(X_train_encoded)

# Evaluate Logistic Regression model on train set
logreg_train_scores = {
    "accuracy": accuracy_score(y_train, logreg_train_preds),
    "precision": precision_score(y_train, logreg_train_preds, average='weighted'),
    "recall": recall_score(y_train, logreg_train_preds, average='weighted'),
    "f1": f1_score(y_train, logreg_train_preds, average='weighted')
}
print("Logistic Regression Scores:")
print("Accuracy:", logreg_train_scores["accuracy"])
print("Precision:", logreg_train_scores["precision"])
print("Recall:", logreg_train_scores["recall"])
print("F1-Score:", logreg_train_scores["f1"])
print()

Logistic Regression Scores:
Accuracy: 0.9323181049069373
Precision: 0.9323620749027023
Recall: 0.9323181049069373
F1-Score: 0.93207665684836



In [ ]:
# Make predictions on the test set
logreg_test_preds = logreg_model.predict(X_test_encoded)

# Evaluate Logistic Regression model on the test set
logreg_test_scores = {
    "accuracy": accuracy_score(y_test, logreg_test_preds),
    "precision": precision_score(y_test, logreg_test_preds, average='weighted'),
    "recall": recall_score(y_test, logreg_test_preds, average='weighted'),
    "f1": f1_score(y_test, logreg_test_preds, average='weighted')
}
print("Logistic Regression Scores on Test Data:")
print("Accuracy:", logreg_test_scores["accuracy"])
print("Precision:", logreg_test_scores["precision"])
print("Recall:", logreg_test_scores["recall"])
print("F1-Score:", logreg_test_scores["f1"])


Logistic Regression Scores on Test Data:
Accuracy: 0.7837837837837838
Precision: 0.7845339895409185
Recall: 0.7837837837837838
F1-Score: 0.7832770876927347


In [ ]:
# Train SVM model
svm_model = SVC()
svm_model.fit(X_train_encoded, y_train)

# Make predictions on train set
svm_train_preds = svm_model.predict(X_train_encoded)

# Evaluate SVM model on train set
svm_train_scores = {
    "accuracy": accuracy_score(y_train, svm_train_preds),
    "precision": precision_score(y_train, svm_train_preds, average='weighted'),
    "recall": recall_score(y_train, svm_train_preds, average='weighted'),
    "f1": f1_score(y_train, svm_train_preds, average='weighted')
}
print("SVM Scores:")
print("Accuracy:", svm_train_scores["accuracy"])
print("Precision:", svm_train_scores["precision"])
print("Recall:", svm_train_scores["recall"])
print("F1-Score:", svm_train_scores["f1"])
print()

SVM Scores:
Accuracy: 0.8451776649746193
Precision: 0.8523938821006966
Recall: 0.8451776649746193
F1-Score: 0.8433773228300224



In [ ]:
# Make predictions on the test set
svm_test_preds = svm_model.predict(X_test_encoded)

# Evaluate SVM model on the test set
svm_test_scores = {
    "accuracy": accuracy_score(y_test, svm_test_preds),
    "precision": precision_score(y_test, svm_test_preds, average='weighted'),
    "recall": recall_score(y_test, svm_test_preds, average='weighted'),
    "f1": f1_score(y_test, svm_test_preds, average='weighted')
}
print("SVM Scores on Test Data:")
print("Accuracy:", svm_test_scores["accuracy"])
print("Precision:", svm_test_scores["precision"])
print("Recall:", svm_test_scores["recall"])
print("F1-Score:", svm_test_scores["f1"])


SVM Scores on Test Data:
Accuracy: 0.7956081081081081
Precision: 0.8068776379808089
Recall: 0.7956081081081081
F1-Score: 0.794308916620712


In [ ]:
# Train Neural Network model
nn_model = MLPClassifier(hidden_layer_sizes=(768,2), max_iter=100, early_stopping=True, validation_fraction=0.2)
nn_model.fit(X_train_encoded, y_train)

# Make predictions on train set
nn_train_preds = nn_model.predict(X_train_encoded)

# Evaluate Neural Network model on train set
nn_train_scores = {
    "accuracy": accuracy_score(y_train, nn_train_preds),
    "precision": precision_score(y_train, nn_train_preds, average='weighted'),
    "recall": recall_score(y_train, nn_train_preds, average='weighted'),
    "f1": f1_score(y_train, nn_train_preds, average='weighted')
}
print("Neural Network Scores:")
print("Accuracy:", nn_train_scores["accuracy"])
print("Precision:", nn_train_scores["precision"])
print("Recall:", nn_train_scores["recall"])
print("F1-Score:", nn_train_scores["f1"])
print()

Neural Network Scores:
Accuracy: 0.37140439932318103
Precision: 0.22752043963769022
Recall: 0.37140439932318103
F1-Score: 0.24584076670682117



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Make predictions on the test set
nn_test_preds = nn_model.predict(X_test_encoded)

# Evaluate Neural Network model on the test set
nn_test_scores = {
    "accuracy": accuracy_score(y_test, nn_test_preds),
    "precision": precision_score(y_test, nn_test_preds, average='weighted'),
    "recall": recall_score(y_test, nn_test_preds, average='weighted'),
    "f1": f1_score(y_test, nn_test_preds, average='weighted')
}
print("Neural Network Scores on Test Data:")
print("Accuracy:", nn_test_scores["accuracy"])
print("Precision:", nn_test_scores["precision"])
print("Recall:", nn_test_scores["recall"])
print("F1-Score:", nn_test_scores["f1"])


Neural Network Scores on Test Data:
Accuracy: 0.3564189189189189
Precision: 0.18824999446621068
Recall: 0.3564189189189189
F1-Score: 0.22231023114773288


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import numpy as np
# Reshape X_train_encoded to add another dimension
X_train_encoded_reshaped = np.expand_dims(X_train_encoded, axis=-1)

In [ ]:
# Train LSTM model
lstm_model = Sequential()
lstm_model.add(Dense(64, input_shape=(768,), activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_encoded_reshaped, y_train, epochs=10, batch_size=32)

# Make predictions on train set
lstm_train_preds = (lstm_model.predict(X_train_encoded_reshaped) > 0.5).astype(int).flatten()

# Evaluate LSTM model on train set
lstm_train_scores = {
    "accuracy": accuracy_score(y_train, lstm_train_preds),
    "precision": precision_score(y_train, lstm_train_preds, average='weighted'),
    "recall": recall_score(y_train, lstm_train_preds, average='weighted'),
    "f1": f1_score(y_train, lstm_train_preds, average='weighted')
}
print("LSTM Scores:")
print("Accuracy:", lstm_train_scores["accuracy"])
print("Precision:", lstm_train_scores["precision"])
print("Recall:", lstm_train_scores["recall"])
print("F1-Score:", lstm_train_scores["f1"])
print()

Epoch 1/10
74/74 [==============================] - 2s 4ms/step - loss: -80.5166 - accuracy: 0.1485
Epoch 2/10
74/74 [==============================] - 0s 4ms/step - loss: -395.4173 - accuracy: 0.1485
Epoch 3/10
74/74 [==============================] - 0s 4ms/step - loss: -975.5187 - accuracy: 0.1485
Epoch 4/10
74/74 [==============================] - 0s 4ms/step - loss: -1839.7329 - accuracy: 0.1485
Epoch 5/10
74/74 [==============================] - 0s 5ms/step - loss: -2989.1458 - accuracy: 0.1485
Epoch 6/10
74/74 [==============================] - 0s 4ms/step - loss: -4413.9785 - accuracy: 0.1485
Epoch 7/10
74/74 [==============================] - 0s 4ms/step - loss: -6115.4375 - accuracy: 0.1485
Epoch 8/10
74/74 [==============================] - 0s 5ms/step - loss: -8082.0576 - accuracy: 0.1485
Epoch 9/10
74/74 [==============================] - 0s 5ms/step - loss: -10309.3975 - accuracy: 0.1485
Epoch 10/10
74/74 [==============================] - 0s 3ms/step
LSTM Scores:
Accurac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Reshape X_test_encoded to add another dimension
X_test_encoded_reshaped = np.expand_dims(X_test_encoded, axis=-1)


In [ ]:
# Make predictions on the test set
lstm_test_preds = (lstm_model.predict(X_test_encoded_reshaped) > 0.5).astype(int).flatten()

# Evaluate the LSTM model on the test set
lstm_test_scores = {
    "accuracy": accuracy_score(y_test, lstm_test_preds),
    "precision": precision_score(y_test, lstm_test_preds, average='weighted'),
    "recall": recall_score(y_test, lstm_test_preds, average='weighted'),
    "f1": f1_score(y_test, lstm_test_preds, average='weighted')
}
print("LSTM Scores on Test Data:")
print("Accuracy:", lstm_test_scores["accuracy"])
print("Precision:", lstm_test_scores["precision"])
print("Recall:", lstm_test_scores["recall"])
print("F1-Score:", lstm_test_scores["f1"])

19/19 [==============================] - 0s 2ms/step
LSTM Scores on Test Data:
Accuracy: 0.13851351351351351
Precision: 0.01918599342585829
Recall: 0.13851351351351351
F1-Score: 0.033703584890528514


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
